In [28]:
! pip install stop_words

You are using pip version 8.0.3, however version 8.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import collections

s1 = "The quick brown fox"
s2 = "Brown fox jumps over the jumps jumps jumps"
s3 = "The the the lazy dog elephant."
s4 = "The the the the the dog peacock lion tiger elephant"

docs = collections.OrderedDict()
docs["s1"] = s1
docs["s2"] = s2
docs["s3"] = s3
docs["s4"] = s4 
for k, v in docs.items():
    print(k,v)
    

s1 The quick brown fox
s2 Brown fox jumps over the jumps jumps jumps
s3 The the the lazy dog elephant.
s4 The the the the the dog peacock lion tiger elephant


In [2]:
len(docs)

4

**Function to make Corpus Matrix**

Function returns a list, 0th element is a 3 dimensional array, each element corresponds to a document in the corpus where the columns are the unique words in that document and the rows are the words in that document (stop words removed).  Second element in list is the column names for each document.

In [3]:
import string
import stop_words
import numpy as np
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer



#Function to make document, word matricies for LDA#
def make_word_matrix(corpus, needToSplit):
    
    #Define stop words
    stopWords = get_stop_words('english')
    
    
    #Initialize stemmer
    p_stemmer = PorterStemmer()

    
    #Define list to store corpus data#
    c = []
    #Define list to store order of words for each document#
    wordOrder = []
    #Define table to remove punctuation
    table = dict.fromkeys(map(ord, string.punctuation))

    M = len(corpus)
    
    #Check to make sure that dictionary isn't empty#
    if M ==0:
        print("Input dictionary is empty")
        return;
    
    removePunc = string.punctuation
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        
        if isinstance(i, str) != True:
            print("Corpus input is not a string")
            return;
        #if the documents in the corpus are contained in a single string
        if needToSplit == 1:
            #Remove punctuation 
            text = corpus[i].translate(table)
            #Splits string by blankspace and goes to lower case#
            words = text.lower().split()
        
        else:
            #Remove punctuation
            for j in range(0, len(removePunc)):
                while removePunc[j] in corpus[i]: 
                    corpus[i].remove(removePunc[j])    
            
            #convert everything to a lower case
            corpus[i] = list(map(lambda x:x.lower(),corpus[i]))
            words = corpus[i]

        #Remove stop words#
        text = [word for word in words if word not in stopWords]
        # stem tokens
        text = [p_stemmer.stem(i) for i in text]
        #Find total number of words in each document#
        N = len(text)

        #Find number of unique words in each document#
        Vwords = list(set(text))
        wordOrder.append(Vwords)

    #Find unique words in the corpus, this is the vocabulary#    
    wordOrder = list(set(x for l in wordOrder for x in l))
    wordOrder = sorted(wordOrder)

    #Find the number of unique words in the corpus vocabulary#
    V = len(wordOrder)
    
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        
        #if the documents in the corpus are contained in a single string
        if needToSplit == 1:
            #Remove punctuation 
            text = corpus[i].translate(table)
            #Splits string by blankspace and goes to lower case#
            words = text.lower().split()
            #Remove stop words#
            text = [word for word in words if word not in stopWords]
            #Stemming
            text = [p_stemmer.stem(i) for i in text]
        else:
            #Remove punctuation
            for j in range(0, len(removePunc)):
                while removePunc[j] in corpus[i]: 
                    corpus[i].remove(removePunc[j])    
            
            #convert everything to a lower case
            corpus[i] = list(map(lambda x:x.lower(),corpus[i]))
            words = corpus[i]
            
            #remove stop words
            text = [word for word in words if word not in stopWords]
            #Stemming
            text = [p_stemmer.stem(i) for i in text]
        #Find total number of words in each document#
        N = len(text)

        #Create matrix to store words for each document#
        wordsMat = np.zeros((N, V))
        count = 0
        for word in text:
            v = wordOrder.index(word)
            wordsMat[count, v] = 1
            count = count + 1
        c.append(wordsMat)

    return [c, wordOrder, M] 

In [4]:
corpusMatrix = make_word_matrix(docs, 1)
corpusMatrix

[[array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
         [ 0.,  0.

**Variational Inference**

In [5]:
import numpy as np
import scipy
from scipy import special

**E-Step:** This function uses variational inference to perform the E step in the EM algorithm to estimate the paramteters in the model.  The output of this function are the matricies gamma and phi, where gamma (k vector) is the Dirichlet paramteters and the matrix phi (N x k, where k is the number of topics) are the multinomial paramters.  See page 1004 of paper for derivation.

In [6]:
def Estep(k, d, alpha, beta, corpusMatrix, tol):    
    
    #storing the total number of words and the number of unique words
    N = corpusMatrix[0][d].shape[0]
    V = corpusMatrix[0][d].shape[1]
    
    #initialize phi and gamma
    oldPhi  = np.full(shape = (N,k), fill_value = 1/k)
    gamma = alpha + N/k
    newPhi = oldPhi
    converge = 0 
    
    
    count = 0
    
    while converge == 0:
        newPhi  = np.zeros(shape = (N,k))
        for n in range(0, N):
            for i in range(0,k):
                newPhi[n,i] = (beta[i, list(corpusMatrix[0][d][n,:]).index(1)])*np.exp(scipy.special.psi(gamma[i]))
        newPhi = newPhi/np.sum(newPhi, axis = 1)[:, np.newaxis] #normalizing the rows of new phi

        for i in range(0,k):
            gamma[i] = alpha[i] + np.sum(newPhi[:, i]) #updating gamma


        criteria = (1/(N*k)*np.sum((newPhi - oldPhi)**2))**0.5
        if criteria < tol:
            converge = 1
        else:
            oldPhi = newPhi
            count = count +1
            converge = 0
    return (newPhi, gamma)

In [7]:
k = 3
np.random.seed(7)
alphaOld = 10*np.random.rand(k)
V = corpusMatrix[0][0].shape[1]
betaOld = np.random.rand(k, V)
betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]

In [8]:
%%time 
for i in range(0,100):
    phi, gamma = Estep(k, 0, alphaOld, betaOld, corpusMatrix, 0.1)

CPU times: user 21.2 ms, sys: 514 µs, total: 21.7 ms
Wall time: 29.2 ms


**Parameter Estimation**

**M Step:** In the E step above, we maximized a lower bound with respect to gamma and phi, and in the M step, for fixed values of these variational parameters, we maximize the lower bound of the log likelihood with repsect to alpha and beta to update these values (combined, these two steps give approximate empirical Bayes estimates for the LDA model).  See pg. 1006 and appendix A.2 for derivation.  

The alphaUpdate() function uses the linear Newton-Rhapson method to update the Dirichlet parameters, alpha, while the Mstep() function maximizes for alpha and beta.

In [9]:
#Update alpha using linear Newton-Rhapson Method#
def alphaUpdate(k, M, alphaOld, gamma, tol):
    h = np.zeros(k)
    g = np.zeros(k)
    alphaNew = np.zeros(k)

    converge = 0
    while converge == 0:
        for i in range(0, k):
            docSum = 0
            for d in range(0, M):
                docSum += scipy.special.psi(gamma[d][i]) - scipy.special.psi(np.sum(gamma[d]))
            g[i] = M*(scipy.special.psi(sum(alphaOld)) - scipy.special.psi(alphaOld[i])) + docSum
            h[i] = M*scipy.special.polygamma(1, alphaOld[i])
        z =  -scipy.special.polygamma(1, np.sum(alphaOld))
        c = np.sum(g/h)/(1/z + np.sum(1/h))
        step = (g - c)/h
        if np.linalg.norm(step) < tol:
            converge = 1
        else:
            converge = 0
            alphaNew = alphaOld + step
            alphaOld = alphaNew

    return alphaNew

In [10]:
def Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol):
    #Calculate beta#
    V = corpusMatrix[0][0].shape[1]
    beta = np.zeros(shape = (k,V))

    for i in range(0,k):
        for j in range(0,V):
            wordSum = 0
            for d in range(0,M):
                Nd = corpusMatrix[0][d].shape[0]
                for n in range(0, Nd):
                    wordSum += phi[d][n,i]*corpusMatrix[0][d][n,j]
            beta[i,j] = wordSum
    #Normalize the rows of beta#
    beta = beta/np.sum(beta, axis = 1)[:, np.newaxis]
    
    ##Update ALPHA##
    alphaNew = alphaUpdate(k, M, alphaOld, gamma, tol)
    return(alphaNew, beta)



**LDA Function:**
Finally, we implement the entire Latent Dirichlet Allocation method in the LDA function, which takes as its arguments k (the number of topics), D (the number of documents in the corpus), a corpus matrix (the output from make_word_matrix above) and a tolerance (which sets the convergence criteria for the while loops).  For each document d, the function runs until the alpha or beta parameters converge, by first running the E step and then the M step for each document separately.  The final values of phi, gamma, alpha and beta are returned for all D documents in a list.

In [16]:
#k = number of topics, D = number of documents#
#corpus matrix is output of make_word_matrix# 
def LDA(k, corpusMatrix, tol, needToSplit):

    
    ##Check for proper input##
    if isinstance(k, int) != True or k <= 0:
        print("Number of topics must be a positive integer")
        return;
    
    if tol <=0:
        print("Convergence tolerance must be positive")
        return;
    
    if needToSplit != 0 and needToSplit != 1:
        print("NeedToSplit argument must be 0 or 1")
        return;
    
    
    
    M = corpusMatrix[2]
    output = []
    
    converge = 0
    #initialize alpha and beta for first iteration
    alphaOld = 10*np.random.rand(k)
    V = corpusMatrix[0][0].shape[1]
    betaOld = np.random.rand(k, V)
    betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
    
    while converge == 0:
        phi = []
        gamma = []
        #looping through the number of documents
        for d in range(0,M): #M is the number of documents
            phiT, gammaT = Estep(k, d, alphaOld, betaOld, corpusMatrix, tol)
            phi.append(phiT)
            gamma.append(gammaT)
            
        alphaNew, betaNew = Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol)
    
        if np.linalg.norm(alphaOld - alphaNew) < tol or np.linalg.norm(betaOld - betaNew) < tol:
            converge =1
        else: 
            converge =0
            alphaOld = alphaNew
            betaOld = betaNew
    output.append([phi, gamma, alphaNew, betaNew])
        
    return output

In [17]:
LDA(3, corpusMatrix, 0.1, 1)

[[[array([[ 0.31630717,  0.07473918,  0.60895365],
          [ 0.70219878,  0.16473784,  0.13306338],
          [ 0.91969551,  0.04354873,  0.03675576]]),
   array([[ 0.6959069 ,  0.1920422 ,  0.1120509 ],
          [ 0.91772006,  0.05111565,  0.03116429],
          [ 0.6411168 ,  0.1039183 ,  0.2549649 ],
          [ 0.6411168 ,  0.1039183 ,  0.2549649 ],
          [ 0.6411168 ,  0.1039183 ,  0.2549649 ],
          [ 0.6411168 ,  0.1039183 ,  0.2549649 ]]),
   array([[ 0.19646656,  0.05598327,  0.74755017],
          [ 0.28706325,  0.17504326,  0.53789349],
          [ 0.16130789,  0.06537213,  0.77331998]]),
   array([[ 0.27104763,  0.17113125,  0.55782111],
          [ 0.17549136,  0.01688476,  0.80762388],
          [ 0.16164125,  0.05066621,  0.78769253],
          [ 0.22631959,  0.02108753,  0.75259288],
          [ 0.14958747,  0.06276942,  0.78764311]])],
  [array([ 7.31364155,  1.56582557,  6.91214363]),
   array([ 9.55353423,  1.94163087,  7.29644564]),
   array([ 6.0202778 ,

In [10]:
#function to return the most probable words
#p is the number of words you want returned for each topic
def mostCommon(beta, wordList, p):
    k = beta.shape[0]
    topicWords = []
    betaDF = pd.DataFrame(beta)
    betaDF.columns = wordList
    for i in range(0, k):
        document = betaDF.loc[i,:]
        document.sort(1, ascending = 0)
        mostCommon = pd.DataFrame(document[0:p])
        topicWords.append(mostCommon)
    return(topicWords)

In [11]:
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health." 
docsTest = collections.OrderedDict()
docsTest["s1"] = doc_a
docsTest["s2"] = doc_b
docsTest["s3"] = doc_c
docsTest["s4"] = doc_d 
docsTest["s5"] = doc_e 
for k, v in docsTest.items():
    print(k,v)
    

s1 Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother.
s2 My mother spends a lot of time driving my brother around to baseball practice.
s3 Some health experts suggest that driving may cause increased tension and blood pressure.
s4 I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better.
s5 Health professionals say that brocolli is good for your health.


In [13]:
cTest = make_word_matrix(docsTest, 1)

**State of the Union Example**

In order to determine the initial performance time of our algorithm, we implemented LDA on a small corpus of eight documents. These documents are each of the State of the Union Adresses delivered by President Clinton during his time in office. These were found in the nltk Python package.

In [18]:
import nltk
#nltk.download("state_union")
from nltk.corpus import state_union

In [19]:
#creating a diciontary of the state of the Union Addresses for Clinton
fileNames = state_union.fileids()
Clinton = fileNames[50:54]

ClintonSOTU = {}
for i in range(0, len(Clinton)):
    ClintonSOTU[Clinton[i].rsplit('.', 1)[0]] = list(nltk.corpus.state_union.words(Clinton[i]))

In [10]:
#should we include the construction of the corpus matrix in LDA function?

In [25]:
%%time
np.random.seed(7)
ClintCorp = make_word_matrix(ClintonSOTU, 0)
Results = LDA(3, ClintCorp, 0.1, 0)

CPU times: user 1min 33s, sys: 213 ms, total: 1min 33s
Wall time: 1min 34s


In [77]:
mostCommon(Results[0][3], ClintCorp[1], 10)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


[                 0
 year      0.027425
 peopl     0.018757
 will      0.015316
 can       0.014768
 us        0.011999
 congress  0.010766
 govern    0.010200
 american  0.009560
 let       0.008880
 give      0.008825,                  1
 work      0.028984
 peopl     0.018815
 s         0.017864
 american  0.016418
 can       0.012685
 america   0.011470
 will      0.010532
 t         0.010124
 time      0.009811
 tax       0.009292,                 2
 must     0.015145
 s        0.013888
 will     0.011184
 world    0.010891
 way      0.009042
 help     0.008930
 deficit  0.008924
 famili   0.008755
 need     0.008169
 say      0.007506]

In [23]:
#pip install --pre line-profiler
#pip install psutil
# pip install memory_profiler

In [26]:
#creating profile files for the construction of the word matrix and LDA for SOTU example
%prun -q -D ClintonWordMatrix.prof make_word_matrix(ClintonSOTU,0)
%prun -q -D ClintonLDA.prof LDA(3, ClintCorp, 0.1, 0)

 
*** Profile stats marshalled to file 'ClintonWordMatrix.prof'. 
 
*** Profile stats marshalled to file 'ClintonLDA.prof'. 


In [27]:
#printing profile results
import pstats
wordMatrixProfile = pstats.Stats('ClintonWordMatrix.prof')
LDAProfile = pstats.Stats('ClintonLDA.prof')
wordMatrixProfile.print_stats()
LDAProfile.print_stats()
pass

Mon Apr 25 13:48:55 2016    ClintonWordMatrix.prof

         1089458 function calls (1089034 primitive calls) in 1.095 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    30666    0.005    0.000    0.008    0.000 <ipython-input-10-d69bc967fb63>:96(<lambda>)
        4    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
     4795    0.001    0.000    0.001    0.000 <ipython-input-10-d69bc967fb63>:70(<genexpr>)
    30210    0.030    0.000    0.058    0.000 /Users/annayanchenko/anaconda3/lib/python3.5/site-packages/nltk/stem/porter.py:470(_step4)
        4    0.006    0.001    0.217    0.054 <ipython-input-10-d69bc967fb63>:102(<listcomp>)
        4    0.007    0.002    0.240    0.060 <ipython-input-10-d69bc967fb63>:61(<listcomp>)
    30210    0.037    0.000    0.073    0.000 /Users/annayanchenko/anaconda3/lib/python3.5/site-packages/nltk/stem/porter.py:252(_step1ab)
        1    0.000    0.000    1.095

**Movie Prediction** 

In addition to text analysis, LDA can be implemented for other problems that have the same structure as a text corpora. In this section, we use LDA in order to predict movies preferred by users of the site MovieLens.com. The goal is to predict another movie that the user likes based on the movies that they have already said that they prefer. The definition of preferred here is that the user rated the movie four or five, where five is the maxiumum possible rating. Users who gave a preferred rating to at least 50 movies were used. A training set of 2,535 users was used to fit the model. The model is then shown all but one of the preferred movies for the users in the test data, and this is used to determine the last movie that the user preferred. Thus, it is possible to evaluate the performance of the model.

In [8]:
#reading in the data
import pandas as pd
trainRatings = pd.read_csv("UserRatingsTraining.csv")
testRatings = pd.read_csv("UserRatingsTest.csv")

In [35]:
#making a dictionary for the training and testing data
trainingUsers = list(set(trainRatings['userID']))
trainingDict = {}
for i in range(0, len(trainingUsers)):
    movies = list(map(str, list(pd.DataFrame(trainRatings[trainRatings['userID'] == trainingUsers[0]])['movieID'])))
    trainingDict[trainingUsers[i]] = movies
    
testUsers = list(set(testRatings['userID']))
testDict = {}
for i in range(0, len(testUsers)):
    movies = list(map(str, list(pd.DataFrame(testRatings[testRatings['userID'] == testUsers[0]])['movieID'])))
    testDict[testUsers[i]] = movies

In [56]:
#running LDA analysis to get parameters for movie training data
movieMatrix = make_word_matrix(trainingDict, 0)
movieOutput = LDA(3, 4, movieMatrix, 0.1, 0)

In [ ]:
#use the above parameters to make predictions for the test set
#hold out one movie randomly, then determine the likelihood of the possible held out movies
#the highest likelihood should be assigned to the last movie, if the model works correctly

## Future Steps

- Compare output to Python package
- Test on corpus in paper
- Model topics on different corpus
- Time and optimize (use Cython, quite slow now)
- Run collaborative filtering
- Compare to Gibbs sampling

## Compare our output to Python Package

https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

In [43]:
#! pip install gensim

In [ ]:
p_stemmer = PorterStemmer()
for i in docs:
    stemmed_tokens = p_stemmer.stem(i)

In [66]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
doc_a = "The quick brown fox"
doc_b = "Brown fox jumps over the jumps jumps jumps"
doc_c =  "The the the lazy dog elephant."
doc_d = "The the the the the dog peacock lion tiger elephant"


# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d]

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, iterations = 1)

In [67]:
print(ldamodel.print_topics(num_topics=3, num_words=4))

[(0, '0.208*jump + 0.122*fox + 0.106*brown + 0.105*dog'), (1, '0.199*jump + 0.128*brown + 0.104*fox + 0.103*eleph'), (2, '0.148*jump + 0.133*dog + 0.131*eleph + 0.108*fox')]


In [69]:
beta =  LDATest(3, corpusMatrix, 0.1, 1)[0][3]

In [70]:
mostCommon(beta, corpusMatrix[1], 4)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


[              0
 jump   0.502213
 dog    0.233400
 brown  0.073339
 lion   0.050304,               1
 fox    0.180265
 jump   0.163142
 brown  0.137126
 dog    0.135596,                 2
 jump     0.239419
 eleph    0.143304
 peacock  0.140478
 lion     0.133684]

In [52]:
corpusMatrix[1]

['brown',
 'dog',
 'eleph',
 'fox',
 'jump',
 'lazi',
 'lion',
 'peacock',
 'quick',
 'tiger']

In [53]:
beta

array([[  4.62560896e-34,   3.33343130e-01,   3.33343094e-01,
          3.30440861e-34,   3.27807481e-36,   3.33313776e-01,
          1.55467573e-14,   1.53680851e-14,   1.26552889e-33,
          5.71817087e-15],
       [  1.09742914e-54,   1.99989427e-01,   1.99989449e-01,
          1.97699403e-54,   1.67679702e-57,   1.84020254e-56,
          2.00007041e-01,   2.00007041e-01,   2.83838287e-51,
          2.00007041e-01],
       [  2.22222222e-01,   4.02204997e-33,   1.14123694e-32,
          2.22222222e-01,   4.44444444e-01,   1.02851237e-35,
          2.11185563e-38,   1.71351548e-38,   1.11111111e-01,
          9.35199488e-40]])

In [68]:
#k = number of topics, D = number of documents#
#corpus matrix is output of make_word_matrix# 
def LDATest(k, corpusMatrix, tol, needToSplit):

    
    ##Check for proper input##
    if isinstance(k, int) != True or k <= 0:
        print("Number of topics must be a positive integer")
        return;
    
    if tol <=0:
        print("Convergence tolerance must be positive")
        return;
    
    if needToSplit != 0 and needToSplit != 1:
        print("NeedToSplit argument must be 0 or 1")
        return;
    
    
    
    M = corpusMatrix[2]
    output = []
    
    converge = 0
    iterations = 0
    #initialize alpha and beta for first iteration
    alphaOld = 10*np.random.rand(k)
    V = corpusMatrix[0][0].shape[1]
    betaOld = np.random.rand(k, V)
    betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
    

    phi = []
    gamma = []
    #looping through the number of documents
    for d in range(0,M): #M is the number of documents
        phiT, gammaT = Estep(k, d, alphaOld, betaOld, corpusMatrix, tol)
        phi.append(phiT)
        gamma.append(gammaT)

    alphaNew, betaNew = Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol)


    converge =0
    alphaOld = alphaNew
    betaOld = betaNew
    iterations += iterations
    output.append([phi, gamma, alphaNew, betaNew])
        
    return output